# Point-wise measures

In [ ]:
import os
os.chdir("../")
import sys
import json
from argparse import Namespace
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import foolbox
from sklearn import metrics
from sklearn.metrics import pairwise_distances as dist
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='paper')

import provable_robustness_max_linear_regions.data as dt
from provable_robustness_max_linear_regions import models
from provable_robustness_max_linear_regions.models import load_model
from robustness_curves import generate_curve_data
from utils import NumpyEncoder

## Calculate robustness curves:
Estimated runtime (if no file with data is present): 2 days

In [ ]:
def load_from_json(file_name):

    if not os.path.exists("res/" + file_name + ".json"):
        return None
    else:
        with open("res/" + file_name + ".json", 'r') as fp:
            loaded_json =  json.load(fp)

            for key in loaded_json.keys():
                loaded_json[key]["x"] = np.array(loaded_json[key]["x"])
                loaded_json[key]["y"] = np.array(loaded_json[key]["y"])

                loaded_json[key]["y"][np.isnan(loaded_json[key]["x"])] = 1.0
                loaded_json[key]["x"] = np.nan_to_num(loaded_json[key]["x"], nan = np.nanmax(loaded_json[key]["x"]))

            return loaded_json
        
def save_to_json(dictionary, file_name):
        
    if not os.path.exists("res"):
        os.makedirs("res")

    with open("res/" + file_name + ".json", 'w') as fp:
        json.dump(dictionary, fp, cls = NumpyEncoder)


training_method_to_model_path = {"ST": "provable_robustness_max_linear_regions/models/plain/2019-02-19 01_20_16 dataset=cifar10 nn_type=cnn_lenet_small p_norm=inf lmbd=0.0 gamma_rb=0.0 gamma_db=0.0 ae_frac=0.0 epoch=100.mat",
                                    "MMR+AT_l_inf": "provable_robustness_max_linear_regions/models/mmr+at/2019-02-17 23_20_04 dataset=cifar10 nn_type=cnn_lenet_small p_norm=inf lmbd=0.1 gamma_rb=3.0 gamma_db=3.0 ae_frac=0.5 epoch=100.mat", 
                                    "MMR+AT_l_2": "provable_robustness_max_linear_regions/models/mmr+at/2019-02-24 10_56_36 dataset=cifar10 nn_type=cnn_lenet_small p_norm=2 lmbd=0.5 gamma_rb=0.15 gamma_db=0.15 ae_frac=0.5 lr=0.001 epoch=100.mat", 
                                    "KW_l_inf": "provable_robustness_max_linear_regions/models/kw/p_norm=inf dataset=cifar10_model=cnn_lenet_small_method=robust_eps=0.007843_checkpoint.mat", 
                                    "KW_l_2": "provable_robustness_max_linear_regions/models/kw/p_norm=2 dataset=cifar10_model=cnn_lenet_small_method=robust_eps=0.1_checkpoint.mat", 
                                    "AT_l_inf": "provable_robustness_max_linear_regions/models/at/2019-02-19 01_20_16 dataset=cifar10 nn_type=cnn_lenet_small p_norm=inf lmbd=0.0 gamma_rb=0.0 gamma_db=0.0 ae_frac=0.5 epoch=100.mat", 
                                    "AT_l_2": "provable_robustness_max_linear_regions/models/at/2019-02-22 02_40_47 dataset=cifar10 nn_type=cnn_lenet_small p_norm=2 lmbd=0.0 gamma_rb=0.0 gamma_db=0.0 ae_frac=0.5 epoch=100.mat",
                                    "MMRUNIV": "experiments/additional_models/mmr_univ_cifar10_gammas_6.0_6.0_lmabdas_1.0_6.0.mat"}

n_points = 10000

_, x_test, _, y_test = dt.get_dataset("cifar10")

x_test = x_test[:n_points]
y_test = y_test[:n_points]

x_test = x_test.reshape(n_points, 1, 32, 32, 3)

model_args = Namespace()
n_test_ex, one, model_args.height, model_args.width, model_args.n_col = x_test.shape
model_args.n_in, model_args.n_out = model_args.height * model_args.width * model_args.n_col, y_test.shape[1]
model_args.n_hs = []
model_args.seed = 1
model_args.nn_type = "cnn"
model_args.dataset = "cifar10"

robustness_curve_data = dict()

for training_method in ["ST", "MMR+AT_l_inf", "MMR+AT_l_2", "KW_l_inf", "KW_l_2", "AT_l_inf", "AT_l_2", "MMRUNIV"]:

    robustness_curve_data[training_method] = load_from_json("rob_curve_data_{}_n_points={}".format(training_method, n_points))

    if not robustness_curve_data[training_method]:
        
        sess = tf.InteractiveSession()
        model, _input, _logits, _ = load_model(sess, model_args, training_method_to_model_path[training_method])
        
        f_model = foolbox.models.TensorFlowModel(_input, _logits, (0,1))
        
        args = Namespace()

        args.inputs = x_test
        args.labels = y_test
        args.f_model = f_model
        args.norms = ["inf"]
        args.save = False
        args.plot = False

        robustness_curve_data[training_method] = generate_curve_data(args)

        save_to_json(robustness_curve_data[training_method], "rob_curve_data_{}_n_points={}".format(training_method, n_points))
        
        tf.reset_default_graph()
        sess.close()

## Print Table Data:

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx, array[idx]

for training_method in ["ST", "AT_l_inf", "KW_l_inf", "MMR+AT_l_inf", "MMRUNIV"]:
    for threshold in [1/255, 4/255, 8/255]:
        nearest_index, nearest_value = find_nearest(robustness_curve_data[training_method]["inf"]["x"], threshold)
        print("dfun=l_infty eps={:.4f} method={} robust error={:.2f}".format(threshold, training_method, robustness_curve_data[training_method]["inf"]["y"][nearest_index]))